# "First Order Motion Model for Image Animation" versión simplificada por RT

**Clonar el repositorio**

In [ ]:
!git clone https://github.com/AliaksandrSiarohin/first-order-model

fatal: destination path 'first-order-model' already exists and is not an empty directory.


In [ ]:
cd first-order-model

/content/first-order-model


**Montar el folder de Google Drive en este Colab**

In [ ]:
# Al ejecutar este bloque, aparecerá un URL. El URL se debe de abrir en el explorador
# y otorgar permisos. Ojo: Si te preocupa la seguridad, mejor hazlo en una cuenta de Google
# que te inventes solo para esto. Y subes las cosas en Google Drive de esa cuenta nueva.

# Cuando apruebes en Google Drive, te dara un codigo, y se debe pegar en el campo de texto
# que aparecera, y dar clic en Enter

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!ls /content/gdrive/MyDrive/AVG/first-order-motion-model

00.mp4	09.png		 doll-01.png  got-02.png     statue-02.png
01.png	10-backward.mp4  doll-02.png  got-03.png     statue-03.png
02.png	10.mp4		 doll-03.png  got-04.png     statue-04.png
03.png	11.png		 doll-04.png  got-05.png     vox-adv-cpk.pth.tar
04.mp4	12.png		 doll-05.png  got-06.png     vox-cpk.pth.tar
05.png	cartoon-02.png	 doll-06.png  got-07.png
06.png	cartoon-03.png	 doll-07.png  got-08.png
07.mkv	cartoon-04.jpg	 doll-08.png  got-09.png
08.mp4	cartoons-01.png  got-01.png   statue-01.png


**Agregar esta carpeta a tu Google Drive: https://drive.google.com/drive/folders/1kZ1gCnpfU0BnpdU47pLM_TQ6RypDDqgw?usp=sharing**

In [ ]:
# Aqui debes cargar el modelo que quieras.
# Por defecto tiene el de caras, pero puedes cambiarlo al de cuerpo, stickers, etc

# Una vez cargado se reutiliza para todas las veces que quieras ejecutar lo de abajo
from demo import load_checkpoints
generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', 
                            checkpoint_path='/content/gdrive/MyDrive/AVG/first-order-motion-model/vox-cpk.pth.tar')

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani

**Cargar video, imagen destino, y animar :D**



In [ ]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

# Aqui se pone la imagen que quieres animar
source_image = imageio.imread('/content/gdrive/MyDrive/AVG/obi_wan.jpg')
# Aqui se pone el video fuente, que tiene el movimiento original
driving_video = imageio.get_reader('/content/gdrive/MyDrive/AVG/video_3.mp4')


# Por defecto la imagen y video se reducen a 256x256.
# Ojo! Revisa que ambos (video e imagen) sean cuadrados para que no batalles.

source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]
    
#HTML(display(source_image, driving_video).to_html5_video())

from demo import make_animation
from skimage import img_as_ubyte

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)
imageio.mimsave('../generated.mp4', [img_as_ubyte(frame) for frame in predictions])

# Mostrar abajo el video, es descargable
HTML(display(source_image, driving_video, predictions).to_html5_video())

100%|██████████| 795/795 [00:32<00:00, 24.57it/s]
